# **L’Algoritme Apriori i el Bàsquet. Utilitzant Machine Learning per a identificar els conjunts de jugadors més eficients i les seves relacions.**

## Alumne: Miquel Villaró Cañizal
## Director: Miguel Ángel Cordobés Aranda 
### Treball de Fi de Grau en Enginyeria Matemàtica en Ciència de Dades
### Curs 2021-2022

# Data pre-processing

In [1]:
# Imports necessaris
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
# Importar els fitxers de dades localment
df = pd.read_csv('Data_AllSeason.csv')
df_F4 = pd.read_csv('Data_F4.csv')
df_RegSeason = pd.read_csv('Data_BeforeF4.csv')
df_Before_Oly = pd.read_csv('Before_OlympiacosF4_.csv')

headers = ['Jugada', 'Base', 'Escorta', 'Aler', 'Alerpivot', 'Pivot', 'Diferencial', 'TipusJugada', 'Exit', 
           'Punts', 'Quart', 'Clutch', 'Rival', 'Terreny', 'Competition']
df.columns = headers
df_F4.columns = headers
df_RegSeason.columns = headers
df_Before_Oly.columns = headers


# Per no tenir en compte les posicions dels jugadors més endavant
df['Players'] = df[['Base', 'Escorta', 'Aler', 'Alerpivot', 'Pivot']].agg(','.join, axis=1) 
df_F4['Players'] = df_F4[['Base', 'Escorta', 'Aler', 'Alerpivot', 'Pivot']].agg(','.join, axis=1) 
df_RegSeason['Players'] = df_RegSeason[['Base', 'Escorta', 'Aler', 'Alerpivot', 'Pivot']].agg(','.join, axis=1) 
df_Before_Oly['Players'] = df_Before_Oly[['Base', 'Escorta', 'Aler', 'Alerpivot', 'Pivot']].agg(','.join, axis=1) 

df.head()

,Jugada,Base,Escorta,Aler,Alerpivot,Pivot,Diferencial,TipusJugada,Exit,Punts,Quart,Clutch,Rival,Terreny,Competition,Players
0,1,Calathes,Higgins,Hayes-Davis,Mirotic,Davies,0,2,1,2,1,0,Alba Berlin,Local,Eurolliga,"Calathes,Higgins,Hayes-Davis,Mirotic,Davies"
1,2,Calathes,Higgins,Hayes-Davis,Mirotic,Davies,2,2,1,3,1,0,Alba Berlin,Local,Eurolliga,"Calathes,Higgins,Hayes-Davis,Mirotic,Davies"
2,3,Calathes,Higgins,Hayes-Davis,Mirotic,Davies,5,2,1,2,1,0,Alba Berlin,Local,Eurolliga,"Calathes,Higgins,Hayes-Davis,Mirotic,Davies"
3,4,Calathes,Higgins,Hayes-Davis,Mirotic,Davies,5,2,1,3,1,0,Alba Berlin,Local,Eurolliga,"Calathes,Higgins,Hayes-Davis,Mirotic,Davies"
4,5,Calathes,Higgins,Hayes-Davis,Mirotic,Davies,8,4,1,0,1,0,Alba Berlin,Local,Eurolliga,"Calathes,Higgins,Hayes-Davis,Mirotic,Davies"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642 entries, 0 to 3641
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Jugada       3642 non-null   int64 
 1   Base         3642 non-null   object
 2   Escorta      3642 non-null   object
 3   Aler         3642 non-null   object
 4   Alerpivot    3642 non-null   object
 5   Pivot        3642 non-null   object
 6   Diferencial  3642 non-null   int64 
 7   TipusJugada  3642 non-null   int64 
 8   Exit         3642 non-null   int64 
 9   Punts        3642 non-null   int64 
 10  Quart        3642 non-null   int64 
 11  Clutch       3642 non-null   int64 
 12  Rival        3642 non-null   object
 13  Terreny      3642 non-null   object
 14  Competition  3642 non-null   object
 15  Players      3642 non-null   object
dtypes: int64(7), object(9)
memory usage: 455.4+ KB


Separar en varis datasets per a fer els anàlisis més específics més endavant.

In [4]:
# Dataset èxit/no èxit
df_exit = df[df['Exit'] == 1]
df_nexit = df[df['Exit'] == 0]

# Dataset fases que acaben en punts (anàlisi completament objectiu)
df_pts = df[df['Punts'] != 0]
df_npts = df[df['Punts'] == 0]

# Dataset (èxit) segons el tipus de fase (1, 2, 3, 4, o 5)
# Fase 1
df_f1 = df[df['TipusJugada'] == 1]
df_f1_exit = df_f1[df_f1['Exit'] == 1]
df_f1_nexit = df_f1[df_f1['Exit'] == 0]

# Fase 2
df_f2 = df[df['TipusJugada'] == 2]
df_f2_exit = df_f2[df_f2['Exit'] == 1]
df_f2_nexit = df_f2[df_f2['Exit'] == 0]

# Fase 3
df_f3 = df[df['TipusJugada'] == 3]
df_f3_exit = df_f3[df_f3['Exit'] == 1]
df_f3_nexit = df_f3[df_f3['Exit'] == 0]

# Fase 4
df_f4 = df[df['TipusJugada'] == 4]
df_f4_exit = df_f4[df_f4['Exit'] == 1]
df_f4_nexit = df_f4[df_f4['Exit'] == 0]

# Fase 5
df_f5 = df[df['TipusJugada'] == 5]
df_f5_exit = df_f5[df_f5['Exit'] == 1]
df_f5_nexit = df_f5[df_f5['Exit'] == 0]

# Datasets en clutch
df_clutch = df[df['Clutch'] == 1]
df_nclutch = df[df['Clutch'] == 0]

# Datasets èxit en clutch
df_exit_clutch = df_clutch[df_clutch['Exit'] == 1]
df_nexit_clutch = df_clutch[df_clutch['Exit'] == 0]

#1. Algoritme depenent de les posicions de joc

In [5]:
# Tenint en compte les posicions dels jugadors
def apriori_posbased(df):
  df = df[['Base', 'Escorta', 'Aler', 'Alerpivot', 'Pivot']]

  # Categorical data
  categorical_cols = ['Base', 'Escorta', 'Aler', 'Alerpivot', 'Pivot'] 

  # One-hot encoding
  df_enc = pd.get_dummies(df, columns = categorical_cols)

  # Construir model
  frq_items = apriori(df_enc, min_support = 0.05, use_colnames = True)
  
  # Recol·lectar les associacions inferides en un dataset
  rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
  rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
  
  return frq_items, rules

Canviar el dataset utilitzat per obtenir el resultat segons què es vulgui analitzar:

- **Èxit**: df_exit
- **No èxit**: df_nexit
- **Anotació**: df_pts
- **No anotació**: df_npts
- **Fase 1 (èxit)**: df_f1_exit
- **Fase 1 (sense èxit)**: df_f1_nexit
- **Fase 2 (èxit)**: df_f2_exit
- **Fase 2 (sense èxit)**: df_f2_nexit
- **Fase 3 (èxit)**: df_f3_exit
- **Fase 3 (sense èxit)**: df_f3_nexit
- **Fase 4 (èxit)**: df_f4_exit
- **Fase 4 (sense èxit)**: df_f4_nexit
- **Fase 5 (èxit)**: df_f5_exit
- **Fase 5 (sense èxit)**: df_f5_nexit
- **Clutch-time (èxit)**: df_exit_clutch
- **Clutch-time (sense èxit)**: df_nexit_clutch

In [6]:
frq_items, rules = apriori_posbased(df_nexit_clutch) #Canviar dataset
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
168,"(Base_Jokubaitis, Aler_Abrines)",(Escorta_Higgins),0.062937,0.237762,0.062937,1.0,4.205882,0.047973,inf
162,"(Base_Jokubaitis, Escorta_Exum)",(Aler_Kuric),0.055944,0.314685,0.055944,1.0,3.177778,0.038339,inf
254,"(Pivot_Sanli, Aler_Exum)",(Escorta_Laprovittola),0.055944,0.426573,0.055944,1.0,2.344262,0.032080,inf
261,"(Alerpivot_Mirotic, Aler_Higgins)",(Escorta_Laprovittola),0.062937,0.426573,0.062937,1.0,2.344262,0.036090,inf
475,"(Aler_Abrines, Alerpivot_Mirotic, Base_Calathe...",(Escorta_Laprovittola),0.062937,0.426573,0.062937,1.0,2.344262,0.036090,inf


# 2. Tenint en compte només la presència dels jugadors a pista

In [7]:
def apriori_normal(df2):

  df = df2.copy()

  # One-hot encoding
  df['Players'] = df['Players'].str.split(",")
  df_enc = pd.concat(
      [
          df.explode("Players")
          .pivot_table(index="Jugada", columns="Players", aggfunc="size", fill_value=0)
      ],
      axis=1,
  )

  # Construir el model
  frq_items = apriori(df_enc, min_support = 0.05, use_colnames = True)
  
  # Recol·lectar les associacions inferides en un dataset
  rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
  rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

  return frq_items, rules

In [8]:
frq_items, rules = apriori_normal(df_nexit_clutch) # Canviar dataset
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
82,"(Jokubaitis, Abrines)",(Higgins),0.062937,0.307692,0.062937,1.0,3.250000,0.043572,inf
172,"(Martinez, Hayes-Davis)",(Davies),0.055944,0.559441,0.055944,1.0,1.787500,0.024647,inf
477,"(Abrines, Mirotic, Calathes, Davies)",(Laprovittola),0.062937,0.587413,0.062937,1.0,1.702381,0.025967,inf
360,"(Exum, Laprovittola, Calathes)",(Mirotic),0.097902,0.762238,0.097902,1.0,1.311927,0.023277,inf
375,"(Higgins, Kuric, Calathes)",(Mirotic),0.062937,0.762238,0.062937,1.0,1.311927,0.014964,inf


# Player score

In [9]:
player_list = ['Calathes', 'Jokubaitis', 'Laprovittola', 'Higgins', 
               'Exum', 'Kuric', 'Abrines', 'Martinez', 
               'Hayes-Davis', 'Mirotic', 'Smits',
               'Oriola', 'Sanli', 'Davies']

In [10]:
# Retorna 4 variables en format diccionari contenint els conjunts de jugadors i el seu score
def scores(df_pos, df_neg, player_list):

  ex_nex_1 = {}
  ex_nex_2 = {}
  ex_nex_3 = {}
  ex_nex_4 = {}

  frq_items_ex, rules_ex = apriori_normal(df_pos)
  frq_items_nex, rules_nex = apriori_normal(df_neg)

  rls_p = pd.DataFrame(rules_ex)
  rls_p["antecedents"] = rls_p["antecedents"].apply(lambda x: list(x)).astype("unicode")
  rls_p["consequents"] = rls_p["consequents"].apply(lambda x: list(x)).astype("unicode")
  rls_p['#Players_ants'] = rls_p['antecedents'].apply(lambda x: len(x.split(',')))
  rls_p['#Players_cons'] = rls_p['consequents'].apply(lambda x: len(x.split(',')))
  rls_p["lift_norm"] = rls_p["lift"]/rls_p['#Players_cons']

  rls_n = pd.DataFrame(rules_nex)
  rls_n["antecedents"] = rls_n["antecedents"].apply(lambda x: list(x)).astype("unicode")
  rls_n["consequents"] = rls_n["consequents"].apply(lambda x: list(x)).astype("unicode")
  rls_n['#Players_ants'] = rls_p['antecedents'].apply(lambda x: len(x.split(',')))
  rls_n['#Players_cons'] = rls_n['consequents'].apply(lambda x: len(x.split(',')))
  rls_n["lift_norm"] = rls_n["lift"]/rls_n['#Players_cons']

  for i in range(len(player_list)):
    ctn_pos = rls_p[rls_p['consequents'].astype(str).str.contains(player_list[i])]
    pos_score_1 = ctn_pos['lift_norm'].sum()

    ctn_neg = rls_n[rls_n['consequents'].astype(str).str.contains(player_list[i])]
    neg_score_1 = ctn_neg['lift_norm'].sum()

    ex_nex_1[player_list[i]] = pos_score_1-neg_score_1


    for j in range(i+1, len(player_list)):
      
      ctn_pos_2 = ctn_pos[ctn_pos['consequents'].astype(str).str.contains(player_list[j])]
      pos_score_2 = ctn_pos_2['lift_norm'].sum()*2

      ctn_neg_2 = ctn_neg[ctn_neg['consequents'].astype(str).str.contains(player_list[j])]
      neg_score_2 = ctn_neg_2['lift_norm'].sum()*2

      name2 = player_list[i]+','+player_list[j]

      total2 = pos_score_2 - neg_score_2
      if total2 != 0:
        ex_nex_2[name2] = total2

      
      for k in range(j+1, len(player_list)):
      
        ctn_pos_3 = ctn_pos_2[ctn_pos_2['consequents'].astype(str).str.contains(player_list[k])]
        pos_score_3 = ctn_pos_3['lift_norm'].sum()*3

        ctn_neg_3 = ctn_neg_2[ctn_neg_2['consequents'].astype(str).str.contains(player_list[k])]
        neg_score_3 = ctn_neg_3['lift_norm'].sum()*3

        name3 = player_list[i]+','+player_list[j]+','+player_list[k]

        total3 = pos_score_3 - neg_score_3
        if total3 != 0:
          ex_nex_3[name3] = total3

        
        for l in range(k+1, len(player_list)):
      
          ctn_pos_4 = ctn_pos_3[ctn_pos_3['consequents'].astype(str).str.contains(player_list[l])]
          pos_score_4 = ctn_pos_4['lift_norm'].sum()*4

          ctn_neg_4 = ctn_neg_3[ctn_neg_3['consequents'].astype(str).str.contains(player_list[l])]
          neg_score_4 = ctn_neg_4['lift_norm'].sum()*4

          name4 = player_list[i]+','+player_list[j]+','+player_list[k]+','+player_list[l]

          total4 = pos_score_4 - neg_score_4
          if total4 != 0:
              ex_nex_4[name4] = total4


  return ex_nex_1, ex_nex_2, ex_nex_3, ex_nex_4


In [11]:
# Computar tots els scores per a totes les fases en grups de 1, 2, 3 i 4 jugadors
general_exit_nexit_1, general_exit_nexit_2, general_exit_nexit_3, general_exit_nexit_4 = scores(df_exit, df_nexit, player_list)
scores_fase1_1, scores_fase1_2, scores_fase1_3, scores_fase1_4 = scores(df_f1_exit, df_f1_nexit, player_list)
scores_fase2_1, scores_fase2_2, scores_fase2_3, scores_fase2_4 = scores(df_f2_exit, df_f2_nexit, player_list)
scores_fase3_1, scores_fase3_2, scores_fase3_3, scores_fase3_4 = scores(df_f3_exit, df_f3_nexit, player_list)
scores_fase4_1, scores_fase4_2, scores_fase4_3, scores_fase4_4 = scores(df_f4_exit, df_f4_nexit, player_list)
scores_fase5_1, scores_fase5_2, scores_fase5_3, scores_fase5_4 = scores(df_f5_exit, df_f5_nexit, player_list)
scores_pts_1, scores_pts_2, scores_pts_3, scores_pts_4 = scores(df_pts, df_npts, player_list)
scores_clutch_1, scores_clutch_2, scores_clutch_3, scores_clutch_4 = scores(df_exit_clutch, df_nexit_clutch, player_list)

In [12]:
# Exemple de scores
scores_fase1_1

{'Abrines': -1.0041293121818802,
 'Calathes': -2.1904988731012764,
 'Davies': -12.552713492127609,
 'Exum': 4.622564033345874,
 'Hayes-Davis': -0.8731115450471556,
 'Higgins': -33.748259007505084,
 'Jokubaitis': -7.005025653299583,
 'Kuric': -3.636290177645627,
 'Laprovittola': 15.11626809520341,
 'Martinez': -0.7420419808042533,
 'Mirotic': -6.378593314212424,
 'Oriola': 0.24939138214208123,
 'Sanli': 0.1265038253485926,
 'Smits': 2.1629376197841204}

In [13]:
# Transformar els diccionaris a dataframes pel seu posterior anàlisi
data = ['General', 'Anotació', 'Fase 1', 'Fase 2', 'Fase 3', 'Fase 4', 'Fase 5', 'Clutch']

df_scores_1 = pd.DataFrame.from_dict([general_exit_nexit_1, scores_pts_1, scores_fase1_1, scores_fase2_1, 
                                      scores_fase3_1, scores_fase4_1, scores_fase5_1, scores_clutch_1])

df_scores_1.insert(0, 'Data', data)

#df_scores_1

In [14]:
df_scores_2 = pd.DataFrame.from_dict([general_exit_nexit_2, scores_pts_2, scores_fase1_2, scores_fase2_2, 
                                      scores_fase3_2, scores_fase4_2, scores_fase5_2, scores_clutch_2])
df_scores_2.insert(0, 'Data', data)

#df_scores_2

In [15]:
df_scores_3 = pd.DataFrame.from_dict([general_exit_nexit_3, scores_pts_3, scores_fase1_3, scores_fase2_3, 
                                      scores_fase3_3, scores_fase4_3, scores_fase5_3, scores_clutch_3])
df_scores_3.insert(0, 'Data', data)

#df_scores_3

In [16]:
df_scores_4 = pd.DataFrame.from_dict([general_exit_nexit_4, scores_pts_4, scores_fase1_4, scores_fase2_4, 
                                      scores_fase3_4, scores_fase4_4, scores_fase5_4, scores_clutch_4])
df_scores_4.insert(0, 'Data', data)

#df_scores_4

In [17]:
# Exemple de dataframe
df_scores_3

,Data,"Calathes,Laprovittola,Exum","Calathes,Laprovittola,Hayes-Davis","Calathes,Laprovittola,Mirotic","Calathes,Laprovittola,Sanli","Calathes,Laprovittola,Davies","Calathes,Higgins,Mirotic","Calathes,Higgins,Davies","Calathes,Exum,Mirotic","Calathes,Kuric,Mirotic",...,"Calathes,Laprovittola,Higgins","Calathes,Abrines,Davies","Jokubaitis,Laprovittola,Kuric","Jokubaitis,Hayes-Davis,Mirotic","Jokubaitis,Hayes-Davis,Davies","Laprovittola,Higgins,Mirotic","Laprovittola,Higgins,Davies","Laprovittola,Abrines,Davies","Kuric,Hayes-Davis,Mirotic","Abrines,Mirotic,Davies"
0,General,1.528160,1.129014,1.228179,0.104390,-0.064508,-0.061187,-0.112133,1.142894,-1.090579,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anotació,1.522505,-1.083533,1.052433,0.006935,0.028446,0.060156,-0.175582,1.137016,-1.066119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fase 1,1.607205,NaN,1.191914,1.497396,0.018684,-2.341071,-1.457143,1.259524,-2.940183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fase 2,-0.039226,NaN,1.001797,-0.048154,1.526348,1.657623,1.497549,-0.117113,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fase 3,1.478107,-2.254545,1.706661,-1.301318,1.355419,1.008869,NaN,1.252356,0.684100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Fase 4,NaN,1.127679,0.135838,1.353353,-0.133782,-0.163293,-0.087018,-1.195934,-1.101019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Fase 5,0.067105,1.072978,-2.979102,0.060887,-0.078103,0.220118,-0.282076,-1.514586,-1.684665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Clutch,3.834117,NaN,3.487535,-0.122598,4.812391,2.036140,3.873523,3.249292,0.626571,...,6.745014,0.701559,-1.166157,3.298724,1.443823,5.830872,4.596781,-1.525172,3.814541,-0.723252


# INTERACTIU
**Tipus de fases**
* Fase 1: 1vs1
* Fase 2: Bloc directe, mà a mà (joc de 2vs2)
* Fase 3: Tir exterior
* Fase 4: Pilota interior, tall (anotació desde pintura)
* Fase 5: Contraatac i transició

**Altres fases del joc**
* E: Èxit
* P: Anotació (objectivitat)
* C: Clutch-time (finals ajustats, jugades en moments i/o partits decisius)

**Llista de jugadors**

Bases: *Calathes, Jokubaitis, Laprovittola*

Exteriors: *Higgins, Exum, Kuric, Abrines, Martinez*

Polivalents (exteriors-interiors): *Hayes-Davis, Mirotic, Smits*

Interiors: *Oriola, Sanli, Davies*

**Informació extra**: A continuació es mostren els millors grups de 3 per a cada tipus de fase.

In [18]:
maxs = df_scores_3.set_index('Data').idxmax(axis = 1)
maxs

Data
General        Laprovittola,Kuric,Mirotic
Anotació       Laprovittola,Kuric,Mirotic
Fase 1         Laprovittola,Kuric,Mirotic
Fase 2            Calathes,Mirotic,Davies
Fase 3      Calathes,Laprovittola,Mirotic
Fase 4        Higgins,Hayes-Davis,Mirotic
Fase 5       Calathes,Hayes-Davis,Mirotic
Clutch            Calathes,Mirotic,Davies
dtype: object

In [25]:
type_ = input('Seleccioni P en cas de joc posicional o N en cas de joc no posicional: ')
data = input('Seleccioni una fase de joc de les indicades prèviament: ')
possible_data = ['1', '2', '3', '4', '5', 'E', 'P', 'C']
possible_type = ['P', 'N']

if data in possible_data and type_ in possible_type:
  if data == '1':
    dataset = df_f1_exit

  elif data == '2':
    dataset = df_f2_exit

  elif data == '3':
    dataset = df_f3_exit

  elif data == '4':
    dataset = df_f4_exit

  elif data == '5':
    dataset = df_f5_exit

  elif data == 'E':
    dataset = df_exit

  elif data == 'C':
    dataset = df_exit_clutch

  elif data == 'P':
    dataset = df_pts
    

  if type_ == 'P':
    print('\nIMPORTANT!\nMàxim 3 jugadors! En cas de no introduir jugador, prèmer ENTER')
    player1 = input('Seleccioni el base de la llista prèvia: ')
    if len(player1) != 0:
      base = 'Base_'+player1
    else:
      base = player1
    
    player2 = input("Seleccioni l'escorta de la llista prèvia: ")
    if len(player2) != 0:
      escorta = 'Escorta_'+player2
    else:
      escorta = player2

    player3 = input("Seleccioni l'aler de la llista prèvia: ")
    if len(player3) != 0:
      aler = 'Aler_'+player3
    else:
      aler = player3

    player4 = input("Seleccioni l'aler-pivot de la llista prèvia: ")
    if len(player4) != 0:
      alerpivot = 'Alerpivot_'+player4
    else:
      alerpivot = player4

    player5 = input("Seleccioni el pivot de la llista prèvia: ")
    if len(player5) != 0:
      pivot = 'Pivot_'+player5
    else:
      pivot = player5

    frq_items, rules = apriori_posbased(dataset)

    a = rules[rules['antecedents'].astype(str).str.contains(base)]
    if len(a) != 0:
      b = a[a['antecedents'].astype(str).str.contains(escorta)]
      if len(b) != 0:
        c = b[b['antecedents'].astype(str).str.contains(aler)]
        if len(c) != 0:
          d = c[c['antecedents'].astype(str).str.contains(alerpivot)]
          if len(d) != 0:
            e = d[d['antecedents'].astype(str).str.contains(pivot)]
            if len(e) != 0:
              print('\nMillors jugadors per maximitzar èxit en fase', data, ':')
              for i in range(len(e)):
                print(list(e['consequents'].iloc[i]))
            else:
              print('Not enough data on this playerset')
          else:
            print('Not enough data on this playerset')
        else:
          print('Not enough data on this playerset')
      else:
        print('Not enough data on this playerset')
    else:
      print('Not enough data on this playerset')

  elif type_== 'N':
    player1 = input('Seleccioni el jugador 1 de la llista prèvia: ')
    player2 = input('Seleccioni el jugador 2 de la llista prèvia: ')
    player3 = input('Seleccioni el jugador 3 de la llista prèvia: ')
    frq_items, rules = apriori_normal(dataset)
    a = rules[rules['antecedents'].astype(str).str.contains(player1)]
    if len(a) != 0:
      b = a[a['antecedents'].astype(str).str.contains(player2)]
      if len(b) != 0:
        c = b[b['antecedents'].astype(str).str.contains(player3)]
        if len(c) != 0:
          print('\nMillors jugadors per maximitzar èxit en fase', data, ':')
          for i in range(len(c)):
            print(list(c['consequents'].iloc[i]))
        else:
          print('Not enough data on this playerset')
      else:
        print('Not enough data on this playerset')
    else:
      print('Not enough data on this playerset')    

else:
  print('Invalid input')

Seleccioni P en cas de joc posicional o N en cas de joc no posicional: P
Seleccioni una fase de joc de les indicades prèviament: E

IMPORTANT!
Màxim 3 jugadors! En cas de no introduir jugador, prèmer ENTER
Seleccioni el base de la llista prèvia: Calathes
Seleccioni l'escorta de la llista prèvia: Laprovittola
Seleccioni l'aler de la llista prèvia: 
Seleccioni l'aler-pivot de la llista prèvia: Mirotic
Seleccioni el pivot de la llista prèvia: 

Millors jugadors per maximitzar èxit en fase E :
['Pivot_Sanli']
['Aler_Exum']
['Aler_Hayes-Davis']


# Prediccions

## Predicció semifinals contra el Real Madrid

In [20]:
# Extreure dades pertinents al partit
df_Madrid = df_RegSeason[(df_RegSeason["Rival"]=="Madrid") | (df_RegSeason["Clutch"]==1)]
df_Madrid_exit = df_Madrid[df_Madrid['Exit'] == 1]
df_Madrid_nexit = df_Madrid[df_Madrid['Exit'] == 0]
df_Madrid_pts = df_Madrid[df_Madrid['Punts'] != 0]
df_Madrid_npts = df_Madrid[df_Madrid['Punts'] == 0]

# Computar scores
Madrid_exit_1, Madrid_exit_2, Madrid_exit_3, Madrid_exit_4 = scores(df_Madrid_exit, df_Madrid_nexit, player_list)
Madrid_pts_1, Madrid_pts_2, Madrid_pts_3, Madrid_pts_4 = scores(df_Madrid_pts, df_Madrid_npts, player_list)

# Convertir a dataframes
Madrid_scores_1 = pd.DataFrame.from_dict([Madrid_exit_1, Madrid_pts_1])
Madrid_scores_2 = pd.DataFrame.from_dict([Madrid_exit_2, Madrid_pts_2])
Madrid_scores_3 = pd.DataFrame.from_dict([Madrid_exit_3, Madrid_pts_3])
Madrid_scores_4 = pd.DataFrame.from_dict([Madrid_exit_4, Madrid_pts_4])

dataM = ['Èxit', 'Anotació']
Madrid_scores_1.insert(0, 'Data', dataM)
Madrid_scores_2.insert(0, 'Data', dataM)
Madrid_scores_3.insert(0, 'Data', dataM)
Madrid_scores_4.insert(0, 'Data', dataM)

#Madrid_scores_1

## Predicció 3r-4t lloc contra Olympiacos

In [21]:
# Extreure les dades pertinents al partit pel 3r i 4t lloc
df_Olympiacos = df_Before_Oly[(df_Before_Oly["Rival"]=="Olympiacos") | (df_Before_Oly["Clutch"]==1)]
df_Olympiacos_exit = df_Olympiacos[df_Olympiacos['Exit'] == 1]
df_Olympiacos_nexit = df_Olympiacos[df_Olympiacos['Exit'] == 0]
df_Olympiacos_pts = df_Olympiacos[df_Olympiacos['Punts'] != 0]
df_Olympiacos_npts = df_Olympiacos[df_Olympiacos['Punts'] == 0]

# Computar scores
Oly_exit_1, Oly_exit_2, Oly_exit_3, Oly_exit_4 = scores(df_Olympiacos_exit, df_Olympiacos_nexit, player_list)
Oly_pts_1, Oly_pts_2, Oly_pts_3, Oly_pts_4 = scores(df_Olympiacos_pts, df_Olympiacos_npts, player_list)

# Convertir en dataframes
Oly_scores_1 = pd.DataFrame.from_dict([Oly_exit_1, Oly_pts_1])
Oly_scores_2 = pd.DataFrame.from_dict([Oly_exit_2, Oly_pts_2])
Oly_scores_3 = pd.DataFrame.from_dict([Oly_exit_3, Oly_pts_3])
Oly_scores_4 = pd.DataFrame.from_dict([Oly_exit_4, Oly_pts_4])

dataO = ['Èxit', 'Anotació']
Oly_scores_1.insert(0, 'Data', dataO)
Oly_scores_2.insert(0, 'Data', dataO)
Oly_scores_3.insert(0, 'Data', dataO)
Oly_scores_4.insert(0, 'Data', dataO)

#Oly_scores_1

# Descàrrega de fitxers

En cas de voler descarregar els fitxers, descomentar les línies de codi pertinents i executar la cel·la:

- Scores temporada sencera:

In [22]:
#df_scores_1.transpose().to_csv('Individual_scores.csv')
#df_scores_2.transpose().to_csv('Pair_scores.csv')
#df_scores_3.transpose().to_csv('Trio_scores.csv')
#df_scores_4.transpose().to_csv('Quad_scores.csv')

#pd.concat([df_scores_1, df_scores_2, df_scores_3], axis = 1).transpose().to_csv('All_scores.csv')

- Predicció contra el Real Madrid abans de la semifinal:

In [23]:
#Madrid_scores_1.transpose().to_csv('Ind_Madrid_scores_BF4.csv')
#Madrid_scores_2.transpose().to_csv('Pair_Madrid_scores_BF4.csv')
#Madrid_scores_3.transpose().to_csv('Trio_Madrid_scores_BF4.csv')
#Madrid_scores_4.transpose().to_csv('Quad_Madrid_scores_BF4.csv')

#pd.concat([Madrid_scores_1, Madrid_scores_2, Madrid_scores_3, Madrid_scores_4], axis = 1).transpose().to_csv('All_Madrid_scores.csv')

- Predicció contra Olympiacos abans del partit pel 3r i 4t lloc:

In [24]:
#Oly_scores_1.transpose().to_csv('Ind_Oly_scores_B34.csv')
#Oly_scores_2.transpose().to_csv('Pair_Oly_scores_B34.csv')
#Oly_scores_3.transpose().to_csv('Trio_Oly_scores_B34.csv')
#Oly_scores_4.transpose().to_csv('Quad_Oly_scores_B34.csv')

#pd.concat([Madrid_scores_1, Madrid_scores_2, Madrid_scores_3, Madrid_scores_4], axis = 1).transpose().to_csv('All_Madrid_scores.csv')